# LSTM in Pytorch

In [1]:
#library imports
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import re
import spacy
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import mean_squared_error

## Basic LSTM in Pytorch with random numbers

In [2]:
#input
x = torch.tensor([[1,2, 12,34, 56,78, 90,80],
                 [12,45, 99,67, 6,23, 77,82],
                 [3,24, 6,99, 12,56, 21,22]])

#### using two different models

In [3]:
model1 = nn.Embedding(100, 7, padding_idx=0)
model2 = nn.LSTM(input_size=7, hidden_size=3, num_layers=1, batch_first=True)

In [4]:
out1 = model1(x)
out2 = model2(out1)

In [5]:
print(out1.shape)
print(out1)

torch.Size([3, 8, 7])
tensor([[[-5.4174e-01, -1.7155e-01, -4.8693e-01,  3.8497e-01, -3.5385e-03,
          -1.0634e+00, -4.1223e-01],
         [-8.6591e-01, -1.3508e+00,  9.4254e-01,  6.4646e-01,  2.9356e-01,
           7.5951e-01, -5.0138e-01],
         [ 1.2837e+00, -2.5886e+00,  1.0039e+00, -9.3060e-01,  1.7693e+00,
          -3.0896e-01,  9.9322e-01],
         [ 1.2048e-02, -8.5139e-01,  1.1775e+00, -1.4380e+00, -1.0230e+00,
           7.4439e-01,  5.6055e-01],
         [-1.3378e+00,  1.4067e+00,  5.3239e-01,  2.6176e-01, -3.1070e-01,
           1.8687e+00,  2.2224e-01],
         [ 1.0795e+00,  8.7718e-01,  9.8972e-02, -9.1531e-01,  1.6382e+00,
           3.6572e-01, -2.4066e-02],
         [ 8.3980e-01,  2.5358e-01, -1.5029e+00,  4.8280e-01, -1.0970e+00,
          -8.3829e-01,  1.0583e+00],
         [-2.7382e-01,  1.1340e+00, -1.0321e+00, -5.3979e-02,  1.3821e+00,
           5.1003e-01,  8.6655e-01]],

        [[ 1.2837e+00, -2.5886e+00,  1.0039e+00, -9.3060e-01,  1.7693e+00,
     

In [6]:
out, (ht, ct) = model2(out1)
print(ht)

tensor([[[ 0.1531,  0.0752, -0.1909],
         [-0.0105,  0.0296, -0.0689],
         [ 0.3678,  0.3573, -0.0085]]], grad_fn=<StackBackward>)


#### using nn.sequential

In [7]:
model3 = nn.Sequential(nn.Embedding(100, 7, padding_idx=0),
                        nn.LSTM(input_size=7, hidden_size=3, num_layers=1, batch_first=True))

In [8]:
out, (ht, ct) = model3(x)
print(out)

tensor([[[-0.0271,  0.1583, -0.0157],
         [-0.0437,  0.1407,  0.0617],
         [-0.0670, -0.1189,  0.0074],
         [-0.0716, -0.0733, -0.0279],
         [-0.1028, -0.5734,  0.0115],
         [-0.1743, -0.2054, -0.0173],
         [ 0.0487, -0.1517,  0.0472],
         [ 0.1487, -0.1625,  0.0339]],

        [[-0.0594, -0.2458, -0.0194],
         [ 0.0157, -0.0203,  0.1743],
         [ 0.0401, -0.2843,  0.0494],
         [ 0.0219, -0.5822, -0.0227],
         [ 0.2021, -0.5198, -0.0525],
         [ 0.1674, -0.6413,  0.0573],
         [ 0.2801, -0.4824,  0.0736],
         [ 0.1395, -0.5459, -0.0813]],

        [[ 0.1195,  0.0531,  0.0961],
         [ 0.0700,  0.0591,  0.0007],
         [ 0.1464, -0.0297,  0.0070],
         [ 0.0657, -0.3028,  0.0430],
         [-0.0225, -0.3713,  0.0294],
         [-0.1062, -0.6546,  0.0147],
         [-0.3348, -0.2389,  0.0165],
         [ 0.0777, -0.1876,  0.1009]]], grad_fn=<TransposeBackward0>)


## Multiclass Text Classification

We are going to predict item ratings based on customer reviews bsed on this dataset from Kaggle:
https://www.kaggle.com/nicapotato/womens-ecommerce-clothing-reviews

Download it and change its name.

In [9]:
#loading the data
reviews = pd.read_csv("reviews.csv")
print(reviews.shape)
reviews.head()

(23486, 11)


,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [10]:
reviews['Title'] = reviews['Title'].fillna('')
reviews['Review Text'] = reviews['Review Text'].fillna('')
reviews['review'] = reviews['Title'] + ' ' + reviews['Review Text']

In [11]:
#keeping only relevant columns and calculating sentence lengths
reviews = reviews[['review', 'Rating']]
reviews.columns = ['review', 'rating']
reviews['review_length'] = reviews['review'].apply(lambda x: len(x.split()))
reviews.head()

,review,rating,review_length
0,Absolutely wonderful - silky and sexy and com...,4,8
1,Love this dress! it's sooo pretty. i happen...,5,62
2,Some major design flaws I had such high hopes ...,3,102
3,"My favorite buy! I love, love, love this jumps...",5,25
4,Flattering shirt This shirt is very flattering...,5,38


In [12]:
#changing ratings to 0-numbering
zero_numbering = {1:0, 2:1, 3:2, 4:3, 5:4}
reviews['rating'] = reviews['rating'].apply(lambda x: zero_numbering[x])

In [13]:
#mean sentence length
np.mean(reviews['review_length'])

60.832921740611425

In [51]:
#tokenization
tok = spacy.load('en_core_web_sm')
def tokenize (text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]') # remove punctuation and numbers
    nopunct = regex.sub(" ", text.lower())
    return [token.text for token in tok.tokenizer(nopunct)]

In [19]:
#count number of occurences of each word
counts = Counter()
for index, row in reviews.iterrows():
    counts.update(tokenize(row['review']))

In [20]:
#deleting infrequent words
print("num_words before:",len(counts.keys()))
for word in list(counts):
    if counts[word] < 2:
        del counts[word]
print("num_words after:",len(counts.keys()))

num_words before: 14137
num_words after: 8262


In [21]:
#creating vocabulary
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [22]:
def encode_sentence(text, vocab2index, N=70):
    tokenized = tokenize(text)
    encoded = np.zeros(N, dtype=int)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length

In [23]:
reviews['encoded'] = reviews['review'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))
reviews.head()

<ipython-input-23-f7d8c1e426bb>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  reviews['encoded'] = reviews['review'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))


,review,rating,review_length,encoded
0,Absolutely wonderful - silky and sexy and com...,3,8,"[[2, 3, 4, 5, 6, 7, 8, 7, 9, 0, 0, 0, 0, 0, 0,..."
1,Love this dress! it's sooo pretty. i happen...,4,62,"[[2, 10, 11, 12, 5, 13, 14, 15, 16, 5, 17, 18,..."
2,Some major design flaws I had such high hopes ...,2,102,"[[54, 55, 56, 57, 17, 58, 59, 60, 61, 62, 11, ..."
3,"My favorite buy! I love, love, love this jumps...",4,25,"[[68, 109, 110, 2, 17, 10, 2, 10, 2, 10, 11, 1..."
4,Flattering shirt This shirt is very flattering...,4,38,"[[122, 123, 11, 123, 52, 92, 122, 19, 124, 125..."


In [24]:
#check how balanced the dataset is
Counter(reviews['rating'])

Counter({3: 5077, 4: 13131, 2: 2871, 1: 1565, 0: 842})

In [25]:
X = list(reviews['encoded'])
y = list(reviews['rating'])
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

#### Pytorch Dataset

In [26]:
class ReviewsDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]

In [27]:
train_ds = ReviewsDataset(X_train, y_train)
valid_ds = ReviewsDataset(X_valid, y_valid)

In [28]:
def train_model(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y, l in train_dl:
            x = x.long()
            y = y.long()
            y_pred = model(x, l)
            optimizer.zero_grad()
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc, val_rmse = validation_metrics(model, val_dl)
        if i % 5 == 1:
            print("train loss %.3f, val loss %.3f, val accuracy %.3f, and val rmse %.3f" % (sum_loss/total, val_loss, val_acc, val_rmse))

def validation_metrics (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    for x, y, l in valid_dl:
        x = x.long()
        y = y.long()
        y_hat = model(x, l)
        loss = F.cross_entropy(y_hat, y)
        pred = torch.max(y_hat, 1)[1]
        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        sum_rmse += np.sqrt(mean_squared_error(pred, y.unsqueeze(-1)))*y.shape[0]
    return sum_loss/total, correct/total, sum_rmse/total

In [29]:
batch_size = 5000
vocab_size = len(words)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=batch_size)

### LSTM with fixed length input

In [30]:
class LSTM_fixed_len(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 5)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

In [31]:
model_fixed =  LSTM_fixed_len(vocab_size, 50, 50)

In [33]:
train_model(model_fixed, epochs=30, lr=0.01)

train loss 1.124, val loss 1.165, val accuracy 0.549, and val rmse 1.129


In [34]:
train_model(model_fixed, epochs=30, lr=0.01)

train loss 1.150, val loss 1.122, val accuracy 0.569, and val rmse 1.325


In [ ]:
train_model(model_fixed, epochs=30, lr=0.01)

### LSTM with variable length input

In [35]:
class LSTM_variable_input(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(0.3)
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 5)
        
    def forward(self, x, s):
        x = self.embeddings(x)
        x = self.dropout(x)
        x_pack = pack_padded_sequence(x, s, batch_first=True, enforce_sorted=False)
        out_pack, (ht, ct) = self.lstm(x_pack)
        out = self.linear(ht[-1])
        return out

In [ ]:
model = LSTM_variable_input(vocab_size, 50, 50)

In [ ]:
train_model(model, epochs=30, lr=0.1)

In [ ]:
train_model(model, epochs=30, lr=0.05)

In [ ]:
train_model(model, epochs=30, lr=0.05)

### LSTM with pretrained Glove word embeddings

Download weights from : https://nlp.stanford.edu/projects/glove/ or type in the browser

In [40]:
def load_glove_vectors(glove_file="./glove.6B.50d.txt"):
    """Load the glove word vectors"""
    word_vectors = {}
    with open(glove_file) as f:
        for line in f:
            split = line.split()
            word_vectors[split[0]] = np.array([float(x) for x in split[1:]])
    return word_vectors

In [41]:
def get_emb_matrix(pretrained, word_counts, emb_size = 50):
    """ Creates embedding matrix from word vectors"""
    vocab_size = len(word_counts) + 2
    vocab_to_idx = {}
    vocab = ["", "UNK"]
    W = np.zeros((vocab_size, emb_size), dtype="float32")
    W[0] = np.zeros(emb_size, dtype='float32') # adding a vector for padding
    W[1] = np.random.uniform(-0.25, 0.25, emb_size) # adding a vector for unknown words 
    vocab_to_idx["UNK"] = 1
    i = 2
    for word in word_counts:
        if word in word_vecs:
            W[i] = word_vecs[word]
        else:
            W[i] = np.random.uniform(-0.25,0.25, emb_size)
        vocab_to_idx[word] = i
        vocab.append(word)
        i += 1   
    return W, np.array(vocab), vocab_to_idx

In [43]:
word_vecs = load_glove_vectors()
pretrained_weights, vocab, vocab2index = get_emb_matrix(word_vecs, counts)

In [44]:
class LSTM_glove_vecs(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim, glove_weights) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embeddings.weight.data.copy_(torch.from_numpy(glove_weights))
        self.embeddings.weight.requires_grad = False ## freeze embeddings
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 5)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

In [45]:
model = LSTM_glove_vecs(vocab_size, 50, 50, pretrained_weights)

In [46]:
train_model(model, epochs=30, lr=0.1)

train loss 1.295, val loss 1.243, val accuracy 0.570, and val rmse 1.357


In [ ]:
train_model(model, epochs=30, lr=0.05)

In [ ]:
train_model(model, epochs=30, lr=0.05)

## Predicting ratings using regression instead of classification

In [47]:
def train_model_regr(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y, l in train_dl:
            x = x.long()
            y = y.float()
            y_pred = model(x, l)
            optimizer.zero_grad()
            loss = F.mse_loss(y_pred, y.unsqueeze(-1))
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss = validation_metrics_regr(model, val_dl)
        if i % 5 == 1:
            print("train mse %.3f val rmse %.3f" % (sum_loss/total, val_loss))

def validation_metrics_regr (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    for x, y, l in valid_dl:
        x = x.long()
        y = y.float()
        y_hat = model(x, l)
        loss = np.sqrt(F.mse_loss(y_hat, y.unsqueeze(-1)).item())
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
    return sum_loss/total

In [48]:
class LSTM_regr(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

In [49]:
model =  LSTM_regr(vocab_size, 50, 50)

In [50]:
train_model_regr(model, epochs=30, lr=0.05)

train mse 1.651 val rmse 1.200


In [ ]:
train_model_regr(model, epochs=30, lr=0.05)